<h1 style="text-align: center; font-size: 36px;">Vanishing Gradients / Overfitting / Save and Load</h1>

##### Student information:
- Name: Tuan Anh NGUYEN
- Email: tuan.nguyen@etu.univ-cotedazur.fr

---

##### Tasks:

- Using Kereas tuner Bayesan approach,
    + find the best hyper-paramters
    + for the « fashion_mnist » dataset
    + Metrics = accuracy
    + And upload your notebook and your model on Moodle

1. Read fashion_mnist dataset
2. Build the model and search best hyper-parameters with keras tuner
3. Fit the best model and evaluate it
4. Save the best model
5. Try to load the best model

---





Import neccessary libraries

In [29]:
from tensorflow import keras
import numpy as np
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import pandas as pd

##### 1. Read fashion_mnist dataset

In [30]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [31]:
# Data normalization
X_train = X_train.reshape(len(X_train), -1)/255 # Reshape dataset and normalize it
X_test = X_test.reshape(len(X_test), -1)/255 # Reshape dataset and normalize it
# Some basic verification about labels
nb_classes = len(np.unique(y_train))
assert np.min(y_train) == 0 and np.max(y_train) == nb_classes-1

##### 2. Build the model and search best hyper-parameters with keras tuner

Build the model

In [32]:
def model_builder(hp):          # Wrap a model in a function
    # Define hyper parameters
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    hp_activation = hp.Choice('activation', ['relu', 'sigmoid'])
    inputs = Input(shape=(X_train.shape[1], ))              # Input layer
    x = Dense(hp_units, activation=hp_activation)(inputs)   # Hidden layer
    outputs = Dense(10, activation='softmax')(x)
    
    # Define the model
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

Build the tuner and search the best parameters

In [33]:
def build_search_params():
    tuner = kt.BayesianOptimization(model_builder,
    objective='val_loss',
    max_trials=10,   # total number of model configurations to test
    seed=None,
    tune_new_entries=True,
    allow_new_entries=True, max_retries_per_trial=0,
    directory='Labs',
    project_name='test_tuner')
    # Search for hyper-parameters
    es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[es])

    return tuner, es

In [34]:
tuner, es = build_search_params()

Reloading Tuner from Labs\test_tuner\tuner0.json


In [35]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
# All hyperpameters are on best_hps.values
for k, v in best_hps.values.items():
    print(f"{k}:{v}")

# Buid the  model with the best hyperparemeters 
best_model = tuner.hypermodel.build(best_hps)
# Get the best model 
best_model= tuner.get_best_models(num_models=1)[0]

units:256
activation:sigmoid


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


##### 3. Fit the best model and evaluate it

In [36]:
def fit_evaluate():
    # fit the best_model (as usual)
    hist = best_model.fit(X_train, y_train, epochs=100, validation_split=0.2,
    callbacks=[es])
    # Babysit the best model (as usual)
    pd.DataFrame({'loss':hist.history['loss'], 'val_loss':hist.history['val_loss']}).plot()
    
    # predict with the best model (as usual)
    y_pred = best_model.predict(X_test)
    # Evaluate the model (as usual)
    eval = best_model.evaluate(X_test, y_test)


##### 4. Save the best model

In [37]:
best_model.export("TuanAnhNGUYEN/model")

INFO:tensorflow:Assets written to: TuanAnhNGUYEN/model\assets


INFO:tensorflow:Assets written to: TuanAnhNGUYEN/model\assets


Saved artifact at 'TuanAnhNGUYEN/model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1738301474064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1738361799600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1738301647760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1738361148288: TensorSpec(shape=(), dtype=tf.resource, name=None)


##### 5. Try to load the best model

In [41]:
def load_new_model():
    layers = keras.layers.TFSMLayer('TuanAnhNGUYEN/model')
    inputs = keras.layers.Input(shape=(X_train.shape[1],))
    outputs = layers(inputs)
    model = keras.models.Model(inputs, outputs)

    return model

In [42]:
new_model = load_new_model()

Honestly, I'm still struggling with loading the new model and using it. Could you send me some feedback of my code:
- Is there any problem with my model?
- Did I export the model in the wrong way? 
- Could you give the way using the saved model? 

Thanks for your feedback.

---

##### Conclusion:
- Through this part, we've learned how to use KerasTuner, avoiding the overfitting or vanishing gradient problem. Also, we know how to save and load the best model.

References:
- Tensorflow keras documentations: https://www.tensorflow.org/api_docs/python/tf/keras/